In [7]:
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
# Load the SQL magic extension
%reload_ext sql
# Set display limit
%config SqlMagic.displaylimit = 20

Found pyproject.toml from 'C:\Users\magsa\Desktop\365_Data_Sience\Project\Movie recommendation system\mini-projects\movie-rec-system'

Settings changed:

Config,value
autopandas,True
feedback,True
displaycon,False
named_parameters,True


In [3]:
# Connect to the DuckDB
%sql duckdb:///../movies_data.duckdb

In [31]:
%%sql
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'movies';


,column_name,data_type
0,adult,BOOLEAN
1,backdrop_path,VARCHAR
2,genre_ids,INTEGER[]
3,id,INTEGER
4,original_language,VARCHAR
5,original_title,VARCHAR
6,overview,VARCHAR
7,popularity,DOUBLE
8,poster_path,VARCHAR
9,release_date,DATE


In [30]:
%%sql
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'genres';

,column_name,data_type
0,id,INTEGER
1,name,VARCHAR


In [29]:
%%sql
-- # Change the data type from BIGINT to INT
ALTER TABLE movies ALTER COLUMN id TYPE INT;
ALTER TABLE movies ALTER COLUMN vote_count TYPE INT;
ALTER TABLE movies ALTER COLUMN genre_ids TYPE INT[];
ALTER TABLE genres ALTER COLUMN id TYPE INT;

,Success
